# ESSE 2220 – Lab 3 Report
## Planet Siren System

**Course:** ESSE 2220  
**Lab Title:** Passive Buzzer with Object-Oriented Planet Tones  
**Date Performed:** 2025-09-26  
**Date Submitted:** 2025-10-06

---

## 1. Names & Group Info
**Group Number:** GROUP 5  
**Members:** Yathharthha Kaushal · Owen Oliver

---

## 2. Circuit Setup

### 2.1 Circuit Photo
![Circuit Image](lab3ciruit.jpg)

**Figure 1** - Lab 3 Passive Buzzer Circuit Setup

### 2.2 Component Connections

**GPIO Pins Used:**
- **Buzzer Pin:** GPIO 13 (BCM mode)
- **Button Pin:** GPIO 18 (BCM mode, with pull-up resistor)

**Wiring Details:**
- **Passive Buzzer:**
  - Positive terminal → GPIO 13
  - Negative terminal → Ground (GND)
  
- **Push Button:**
  - One terminal → GPIO 18
  - Other terminal → Ground (GND)
  - Internal pull-up resistor enabled in software (GPIO.PUD_UP)

**Safety Considerations:**
- Used BCM GPIO numbering for hardware independence across different Raspberry Pi models
- Pull-up resistor configuration prevents floating input on button pin
- PWM (Pulse Width Modulation) used to control buzzer frequency safely
- Proper ground connection shared between components to prevent voltage spikes
- GPIO cleanup implemented to release resources and prevent pin conflicts

**Pin Configuration Rationale:**
- GPIO 13 supports hardware PWM for smooth frequency generation
- GPIO 18 chosen for button input to avoid conflict with PWM operations
- BCM mode selected for portability across different Pi versions

---

## 3. Code and Demo

### 3.1 Main Program (main.py)

```python
import RPi.GPIO as GPIO
import planet_tones
import time
import math

buzzerPin = 13        # define the buzzerPin
buttonPin = 18        # define the buttonPin
planetObject = None

def setup():
    """
    Initialize GPIO configuration for buzzer and button.
    Sets BCM mode, configures buzzer as PWM output, and button with pull-up resistor.
    """
    global p
    GPIO.setmode(GPIO.BCM)          # Use BCM GPIO Numbering for hardware independence
    GPIO.setup(buzzerPin, GPIO.OUT) # Set buzzer pin to OUTPUT mode
    GPIO.setup(buttonPin, GPIO.IN, pull_up_down=GPIO.PUD_UP)  # Set button to INPUT with pull-up
    p = GPIO.PWM(buzzerPin, 1)      # Initialize PWM on buzzer pin with 1 Hz base frequency
    p.start(0)                      # Start PWM with 0% duty cycle (off)

def stopAlerter():
    """
    Stop the buzzer PWM signal.
    Halts the alertor sound.
    """
    p.stop()

def loop():
    """
    Main loop monitoring button state.
    When button pressed (LOW), activates alertor with planet-specific tones.
    When button released (HIGH), stops alertor.
    """
    while True:
        if GPIO.input(buttonPin) == GPIO.LOW:  # Button pressed (active low due to pull-up)
            alertor()
            print('alertor turned on >>>')
        else:                                   # Button released
            stopAlerter()
            print('alertor turned off <<<')

def alertor():
    """
    Generate planet-specific siren sound using trigonometric waveform.
    Sweeps through 361 degrees of sine wave to create varying frequency tone.
    Frequency calculated using planet's base tone and depth modulated by sine wave.
    """
    p.start(50)                     # Start PWM at 50% duty cycle for audible sound
    for x in range(0, 361):         # Sweep through full sine wave cycle (0-360 degrees)
        # Configurable waveform parameters
        trigVar = "sin"             # Waveform type: "sin", "tan", or "cos"
        coefficient = 5             # Amplitude multiplier for trigonometric function
        degree = 0.5                # Power factor to modify waveform shape
        
        # Calculate frequency based on selected trigonometric function
        if(trigVar == "sin"):
            # Convert degrees to radians, apply power and coefficient
            sinVal = math.sin(coefficient * math.pow((x * (math.pi / 180.0)), degree))
            toneVal = planetObject.getCalculatedTone(sinVal)
        elif(trigVar == "tan"):
            tanVal = math.tan(coefficient * math.pow((x * (math.pi / 180.0)), degree))
            toneVal = planetObject.getCalculatedTone(tanVal)
        elif(trigVar == "cos"):
            cosVal = math.cos(coefficient * math.pow((x * (math.pi / 180.0)), degree))
            toneVal = planetObject.getCalculatedTone(cosVal)
        
        p.ChangeFrequency(toneVal)  # Update PWM frequency to new tone value
        time.sleep(0.001)           # Small delay for smooth frequency transition

def destroy():
    """
    Cleanup function to safely release GPIO resources.
    Turns off buzzer and releases all GPIO pins.
    """
    GPIO.output(buzzerPin, GPIO.LOW)     # Turn off buzzer
    GPIO.cleanup()                       # Release GPIO resource

if __name__ == '__main__':               # Program entrance
    print('Program is starting...')
    
    # Prompt user to select a planet
    planet = input("Enter a planet (Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, Neptune): ").strip().lower()
    
    # Dictionary mapping planet names to their corresponding class constructors
    planet_classes = {
        "mercury": planet_tones.Mercury,
        "venus": planet_tones.Venus,
        "earth": planet_tones.Earth,
        "mars": planet_tones.Mars,
        "jupiter": planet_tones.Jupiter,
        "saturn": planet_tones.Saturn,
        "uranus": planet_tones.Uranus,
        "neptune": planet_tones.Neptune
    }
    
    # Initialize the selected planet object
    if planet in planet_classes:
        selected_planet = planet_classes[planet]()  # Instantiate planet class
        base, depth = selected_planet.get_tone()    # Get planet's tone parameters
        planetObject = selected_planet              # Store in global for alertor function
        print(f"Selected Planet: {selected_planet.name}, Base Frequency: {base} Hz, Depth: {depth} Hz")
    else:
        print("Invalid planet name. Please choose from the list.")
        exit(1)

    setup()                         # Initialize GPIO configuration
    try:
        loop()                      # Enter main monitoring loop
    except KeyboardInterrupt:       # Handle Ctrl+C gracefully
        destroy()                   # Cleanup GPIO resources
```


### 3.2 Planet Tones Module (planet_tones.py)
```python
class Planet:
    """
    Base class representing a celestial body with unique audio signature.
    Stores planet name, base frequency, and modulation depth for tone generation.
    """

    def __init__(self, name, base, depth):
        """
        Initialize planet with audio characteristics.
        
        Parameters:
        - name (str): Planet name
        - base (int): Base frequency in Hz (center tone)
        - depth (int): Modulation depth in Hz (frequency variation range)
        """
        self.name = name
        self.base = base
        self.depth = depth

    def get_tone(self):
        """
        Retrieve planet's tone parameters.
        
        Returns:
        - list: [base frequency, depth] in Hz
        """
        return [self.base, self.depth]

    def getCalculatedTone(self, sinVal):
        """
        Calculate instantaneous frequency using modulation value.
        
        Parameters:
        - sinVal (float): Modulation value from trigonometric function (-1 to 1)
        
        Returns:
        - float: Instantaneous frequency in Hz
        
        Formula: frequency = base + (sinVal × depth)
        Example: For Earth at sinVal=0.5: 2000 + (0.5 × 500) = 2250 Hz
        """
        return self.base + sinVal * self.depth

class Mercury(Planet):
    """
    Mercury tone: Highest pitch, smallest modulation.
    Base: 1500 Hz, Depth: 300 Hz
    Range: 1200-1800 Hz
    """
    def __init__(self):
        super().__init__("Mercury", 1500, 300)

class Venus(Planet):
    """
    Venus tone: High pitch with moderate modulation.
    Base: 1800 Hz, Depth: 400 Hz
    Range: 1400-2200 Hz
    """
    def __init__(self):
        super().__init__("Venus", 1800, 400)

class Earth(Planet):
    """
    Earth tone: Medium pitch, balanced modulation.
    Base: 2000 Hz, Depth: 500 Hz
    Range: 1500-2500 Hz
    """
    def __init__(self):
        super().__init__("Earth", 2000, 500)

class Mars(Planet):
    """
    Mars tone: Medium-low pitch with deeper modulation.
    Base: 2200 Hz, Depth: 600 Hz
    Range: 1600-2800 Hz
    """
    def __init__(self):
        super().__init__("Mars", 2200, 600)

class Jupiter(Planet):
    """
    Jupiter tone: Low pitch with strong modulation.
    Base: 2500 Hz, Depth: 700 Hz
    Range: 1800-3200 Hz
    """
    def __init__(self):
        super().__init__("Jupiter", 2500, 700)

class Saturn(Planet):
    """
    Saturn tone: Lower pitch with increasing modulation.
    Base: 2700 Hz, Depth: 800 Hz
    Range: 1900-3500 Hz
    """
    def __init__(self):
        super().__init__("Saturn", 2700, 800)

class Uranus(Planet):
    """
    Uranus tone: Very low pitch with wide modulation.
    Base: 2900 Hz, Depth: 900 Hz
    Range: 2000-3800 Hz
    """
    def __init__(self):
        super().__init__("Uranus", 2900, 900)

class Neptune(Planet):
    """
    Neptune tone: Lowest base pitch, widest modulation range.
    Base: 3100 Hz, Depth: 1000 Hz
    Range: 2100-4100 Hz
    """
    def __init__(self):
        super().__init__("Neptune", 3100, 1000)
```

---

### 3.3 Demo Verification

**Demo completed:** ✔️ Completed  

---

## 4. Observations

### 4.1 Planet Tone Characteristics

Each planet has a unique audio signature defined by two parameters:

| Planet  | Base Frequency (Hz) | Depth (Hz) | Frequency Range (Hz) | Auditory Character |
|---------|---------------------|------------|----------------------|--------------------|
| Mercury | 1500 | 300 | 1200 - 1800 | High-pitched, tight variation |
| Venus   | 1800 | 400 | 1400 - 2200 | High pitch, moderate sweep |
| Earth   | 2000 | 500 | 1500 - 2500 | Balanced mid-range tone |
| Mars    | 2200 | 600 | 1600 - 2800 | Deeper with wider range |
| Jupiter | 2500 | 700 | 1800 - 3200 | Low pitch, dramatic sweep |
| Saturn  | 2700 | 800 | 1900 - 3500 | Lower tone, broad modulation |
| Uranus  | 2900 | 900 | 2000 - 3800 | Very low, extensive range |
| Neptune | 3100 | 1000 | 2100 - 4100 | Lowest base, widest sweep |

### 4.2 Siren Behavior

**Waveform Generation:**
- The siren uses a sine wave modulation pattern that sweeps through 361 steps (0° to 360°)
- Frequency varies smoothly according to: `frequency = base + sin(coefficient × degree^power) × depth`
- Default configuration: `coefficient = 5`, `power = 0.5`, waveform = sine

**Button Interaction:**
- Button press (LOW signal): Activates continuous siren sound
- Button release (HIGH signal): Immediately stops siren
- No debouncing needed due to rapid loop execution and continuous playback

**Sound Pattern:**
- Each complete siren cycle takes approximately 361ms (361 steps × 1ms/step)
- Frequency smoothly rises and falls creating distinctive "woo-woo" effect
- Different planets produce distinctly recognizable tones due to base/depth variations


---

## 5. Analysis

### 5.1 Object-Oriented Design Explanation

#### The Planet Class

**What the class stores:**
- `name` (str): The planet's name for identification
- `base` (int): Base frequency in Hz - the center point of the tone
- `depth` (int): Modulation depth in Hz - determines how much the frequency varies

**What the methods do:**

**`__init__(self, name, base, depth)`**
- **Purpose:** Constructor that initializes a planet object with its unique audio characteristics
- **Parameters:** planet name, base frequency, and modulation depth
- **Does:** Stores all three values as instance variables
- **Returns:** New Planet instance

**`get_tone(self)`**
- **Purpose:** Accessor method to retrieve planet's tone parameters
- **Parameters:** None (uses self)
- **Does:** Returns the base frequency and depth as a list
- **Returns:** `[base, depth]` list
- **Usage:** Called in main program to display planet information

**`getCalculatedTone(self, sinVal)`**
- **Purpose:** Calculate instantaneous frequency based on modulation value
- **Parameters:** `sinVal` - a value between -1 and 1 from sine/cos/tan function
- **Does:** Multiplies sinVal by depth and adds to base frequency
- **Returns:** Float representing the frequency in Hz at that moment
- **Usage:** Called 361 times per siren cycle to generate varying tones

**How it's used in the loop:**

1. **Initialization Phase:**
   ```python
   selected_planet = planet_classes[planet]()  # Create planet object (e.g., Earth())
   planetObject = selected_planet               # Store globally
   ```

2. **In the alertor() function:**
   ```python
   for x in range(0, 361):  # For each degree in the cycle
       sinVal = math.sin(5 * math.pow((x * (math.pi / 180.0)), 0.5))  # Calculate modulation
       toneVal = planetObject.getCalculatedTone(sinVal)                # Get frequency
       p.ChangeFrequency(toneVal)                                      # Apply to buzzer
   ```

3. **Step-by-step (Earth at x=90°):**
   - Convert to radians: 90° × (π/180) = 1.571
   - Apply power: (1.571)^0.5 = 1.254
   - Multiply coefficient: 5 × 1.254 = 6.270
   - Calculate sine: sin(6.270) ≈ 0.156
   - Earth's base = 2000, depth = 500
   - Final frequency: 2000 + (0.156 × 500) = 2078 Hz

### 5.2 Why Object-Oriented Programming?

**Advantages of using classes for this lab:**

1. **Code Reusability:**
   - Single `Planet` class serves as template for all 8 planets
   - Subclasses (Mercury, Venus, etc.) inherit all methods from parent
   - No need to duplicate `get_tone()` and `getCalculatedTone()` logic

2. **Maintainability:**
   - To add a new planet, create one new class with just 3 lines of code
   - To change tone calculation formula, modify only `getCalculatedTone()` method
   - Clear separation between planet data and tone generation logic

3. **Extensibility:**
   - Easy to add new methods (e.g., `getVisualColor()`, `getOrbitSpeed()`)
   - Could add planet-specific waveform overrides in subclasses
   - Supports future features without restructuring existing code

4. **Data Organization:**
   - Related data (name, base, depth) grouped together logically
   - Prevents errors from mismatched parameter passing
   - Self-documenting code structure

5. **Polymorphism:**
   - All planet objects share same interface (`get_tone()`, `getCalculatedTone()`)
   - `planetObject` can be any planet type, alertor() works the same way
   - Easy to switch planets without changing main loop code

**Alternative without classes would require:**
- Separate variables or dictionaries for each planet's parameters
- Manual frequency calculation in main loop
- More complex conditional logic to handle different planets
- Higher risk of inconsistent implementations

### 5.3 Program Flow

1. **Startup:** User selects planet from 8 options
2. **Initialization:** Planet object created with specific base/depth values
3. **GPIO Setup:** Buzzer configured for PWM, button configured with pull-up
4. **Main Loop:** Continuously monitors button state
5. **Alertor Activation:** When button pressed, sweeps through 361-step sine wave
6. **Frequency Modulation:** Each step calculates new frequency using planet's tone formula
7. **Cleanup:** On exit, safely releases GPIO resources

---

## 6. Bonus (Optional)

### 6.1 Alternative Waveform Patterns

- Tried different trigonometric functions: `tan` and `cos`, and also tried with `exponential domains` within the trig functions, as well as with different `coefficients`.
---
